In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import seaborn as sns
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
y = np.zeros(600)
y[:300] = 0
y[300:600] = 1 # 1 = +ve

In [ ]:
#base 64 to image cvt

In [ ]:
X = np.zeros((600,256,256,3))

#normal first

for i in range(300):
  img = cv2.imread(f"/content/drive/MyDrive/Covid19/Newron Covid 19/Normal_Large/Normal-{i+1}.png") #cvt to greyscale by 0....Ada path
  img = cv2.resize(img, (256,256))
  X[i] = img.copy()

for i in range(300):
  img = cv2.imread(f"/content/drive/MyDrive/Covid19/Newron Covid 19/Covid_Large/COVID-{i+1}.png") #cvt to greyscale by 0....Ada path
  img = cv2.resize(img, (256,256))
  X[i+100] = img.copy() # storing after 100, first 100= normal CXR

X = X/255.0

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from tensorflow import keras
from keras import layers

data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

In [ ]:
base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(256, 256, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(256, 256, 3))
l = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
l = scale_layer(l)

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
l = base_model(l, training=False)
l = keras.layers.GlobalAveragePooling2D()(l)
l = keras.layers.Dropout(0.2)(l)  # Regularize with dropout
outputs = keras.layers.Dense(1)(l)
model = keras.Model(inputs, outputs)

model.summary()

83683744/83683744 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 xception (Functional)       (None, 8, 8, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0      

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 20
model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/20
15/15 [==============================] - 16s 337ms/step - loss: 0.6959 - binary_accuracy: 0.4958 - val_loss: 0.6913 - val_binary_accuracy: 0.5167
Epoch 2/20
15/15 [==============================] - 3s 210ms/step - loss: 0.6926 - binary_accuracy: 0.4958 - val_loss: 0.6927 - val_binary_accuracy: 0.5167
Epoch 3/20
15/15 [==============================] - 3s 212ms/step - loss: 0.6871 - binary_accuracy: 0.4958 - val_loss: 0.6885 - val_binary_accuracy: 0.5167
Epoch 4/20
15/15 [==============================] - 3s 212ms/step - loss: 0.6903 - binary_accuracy: 0.4958 - val_loss: 0.6888 - val_binary_accuracy: 0.5167
Epoch 5/20
15/15 [==============================] - 3s 212ms/step - loss: 0.6844 - binary_accuracy: 0.4958 - val_loss: 0.6862 - val_binary_accuracy: 0.5167
Epoch 6/20
15/15 [==============================] - 3s 212ms/step - loss: 0.6886 - binary_accuracy: 0.4958 - val_loss: 0.6844 - val_binary_accuracy: 0.5167
Epoch 7/20
15/15 [==============================] - 3s 213ms/st

In [ ]:
def fine_tune(model, X_train, y_train, X_test, y_test, epochs = 10,learning_rate = 1e-5):
  base_model.trainable = True
  model.summary()

  model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
  )

  epochs = 10
  model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

In [ ]:
fine_tune(model,X_train, y_train, X_test, y_test)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 xception (Functional)       (None, 8, 8, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

In [ ]:
# Scoring
predictions = (model.predict(X_test) > 0.5).astype("int32")
predictions = predictions.reshape(1,-1)[0]
print(classification_report(y_test, predictions))

# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, predictions)
auc = roc_auc_score(y_test, predictions)
print('AUC: %.3f' % auc)

4/4 [==============================] - 1s 169ms/step
              precision    recall  f1-score   support

         0.0       0.78      1.00      0.88        62
         1.0       1.00      0.71      0.83        58

    accuracy                           0.86       120
   macro avg       0.89      0.85      0.85       120
weighted avg       0.89      0.86      0.85       120

AUC: 0.853
